# Chain

- 여러 컴포넌트들을 논리적 순서대로 연결하여 복잡한 작업을 수행하는 구조로 복잡한 AI 작업을 체계적이고 효율적으로 구현할 수 있게 해준다.
- 기본 개념
    - 일련의 작업을 구성하는 여러 개별 컴포넌트들을 정의된 순서대로 실행시킨다.
    - 단일 API 호출을 넘어 여러 호출을 논리적 순서로 연결 가능하다.
    - 복잡한 작업을 작은 단계로 분해하여 순차적으로 처리할 수 있다.

- Langchain은 `off-the-shelf chains` 방식과 `LCEL(Langchain Expression Language)`  두가지 방식이 있다.
  - off-the-shelf chains 방식
    - 미리정의된 Chain 클래스를 사용해 체인을 구성하는 방식
    - Langchain의 초기 방식으로 대부분의 class들이 deprecated 되었다.
  - LECL 방식
    - 표현식을 이용해 체인을 구성하는 방식이다.
    - 현재 LangChain은 LCEL(LangChain Expression Language)을 중심으로 발전하고 있다


# Off-the-shelf chains 예제

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from pprint import pprint

prompt_template = PromptTemplate(
    template = "{item}에 어울리는 이름 {count}개를 만들어 주세요."
    
)
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=1.0
)

# 순서
## 1. prompt생성 -> 2. LLM 요청
prompt = prompt_template.invoke({"item":"스마트폰", "count":5})
# prompt.text
result = model.invoke(prompt)
print(result)

content='물론입니다! 스마트폰에 어울리는 이름 5개를 제안드리겠습니다.\n\n1. **스마트엣지** (SmartEdge)\n2. **픽셀래스팅** (PixelLasting)\n3. **모바일파노라마** (MobilePanorama)\n4. **에코플렉스** (EchoPlex)\n5. **퓨처링크** (FutureLink)\n\n이 이름들이 구매자의 관심을 끌 수 있기를 바랍니다!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 22, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None} id='run-d5331775-9448-4e46-8a87-c861be2c69ca-0' usage_metadata={'input_tokens': 22, 'output_tokens': 102, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [3]:
from langchain import LLMChain

chain = LLMChain(
    prompt = prompt_template,
    llm = model
    # , output_parser=OutputParser객체
)
# 입력 : prompt_template의 전달할 값. 
# -> chain(prompt_template 
# -> LLM -> 출력 : LLM의 결과

result = chain.invoke({"item":"가방", "count":3})


C:\Users\Playdata\AppData\Local\Temp\ipykernel_11428\2068059606.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [4]:
print(result)

{'item': '가방', 'count': 3, 'text': '물론입니다! 가방에 어울리는 이름 세 개를 제안해 드릴게요:\n\n1. **스타일리쉬 캐리** (Stylish Carry) - 세련된 느낌을 주는 이름으로, 어떤 모임에서도 잘 어울릴 것 같습니다.\n2. **모던 미니멀** (Modern Minimal) - 심플하면서도 현대적인 디자인의 가방에 잘 어울리는 이름입니다.\n3. **어반 백팩** (Urban Backpack) - 도시 생활에 적합한 실용적인 느낌의 백팩에 적합한 이름입니다.\n\n도움이 되었길 바랍니다!'}


In [5]:
# LCEL
chain2 = prompt_template | model
result2 = chain2.invoke({"item":"컴퓨터", "count":5})

In [6]:
print(result2.content)

물론입니다! 컴퓨터에 어울리는 이름 5개를 제안해 드릴게요:

1. **코드마스터** (CodeMaster)
2. **데이터나비** (DataNavi)
3. **서지온** (Surgeon)
4. **알고리즘아리** (AlgorithmAri)
5. **네오컴** (NeoCom)

이 이름들이 마음에 드시길 바랍니다!


# LCEL (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 **체인(Chain)을 더욱 효율적으로 구현하기 위해 도입된 **선언적 방식의 체인(chain) 구성 언어**이다.
- `|` 연산자를 이용해 선언적 방법으로 Chain을 만든다.
- [Runnable](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html) type의 component들이 chain에 포함될 수있다.
    - `|` 연산자를 이용해 Runnable들을 연결한다.
    - chain이 실행되면 각 Runnable의 invoke() 메소드가 실행된다. 그리고 invoke()의 리턴값을 다음 Runnable의 invoke()에 전달해서 실행시킨다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/expression_language/interface)
        - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## Runnable
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 체인(chain) 형태로 연결하여 복잡한 작업을 수행할 수 있게 한다.
- Chain을 구성하는 class들은 Runnable의 하위 클래스로 구현한다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 요청)을 수행하는 독립적인 컴포넌트이다.
    - LangChain의 다양한 컴포넌트(PromptTemplate, LLM, OutputParser 등)들이 Runnable을# Runnable
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 체인(chain) 형태로 연결하여 복잡한 작업을 수행할 수 있게 한다.
- Chain을 구성하는 class들은 Runnable의 하위 클래스로 구현한다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 요청)을 수행하는 독립적인 컴포넌트이다.
    - LangChain의 다양한 컴포넌트(PromptTemplate, LLM, OutputParser 등)들이 Runnable을 상속받아 구현된다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 파이프라인처럼 체인(순차적으로 실행되는 작업들을 연결한 것)을 구성할 수 있으며, `|` 연산자를 사용해 간단히 연결 가능하다.
    - 입력과 출력 형식을 통일해서 컴포넌트를 매끄럽게 연결한다
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 디버깅 및 유지보수가 용이하다.
    - 복잡한 작업을 작은 단위로 나누어 관리할 수 있다.
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
- `invoke()`: 입력 데이터를 처리하여 결과를 반환.
- `batch()`: 여러 입력 데이터들을 한 번에 처리.
- `stream()`: 스트리밍 방식으로 응답 반환.
- `ainvoke()`: 비동기 호출 지원.

### Runnable의 주요 구현체
- **`RunnablePassThrough`**
    - 입력데이터를 다음 chain으로 그대로 전달하거나, 필요에 따라 추가적인 key-value 쌍을 더해서 전달한다. 
- **`RunnableParallel`**
    - 여러 Runnable을 병렬로 실행하고 결과들을 합쳐서 다음 chain으로 전달한다.`**
- **`RunnableLambda`**
    - 일반 함수나 lambda 함수를 Runnable로 만들 때 사용.

## 사용자 함수를 chain으로 정의
- 임의의 함수를 Runnable로 정의 할 수있다.
    - LangChain 에서 제공하지 않는 기능을 Chain으로 만들 때 유용한다.
- LangChain에서는 Runnable로 사용되는 사용자 정의 함수를 **Runnable Lambda** 라고 한다.
- 함수를 Runnable 로 명시하는데는 다음 두가지 방법이 있다.
1. `RunnableLambda` 이용
   - `RunnableLambda(함수)`
3. `@chain` 데코레이터 이용
   - ```python
     @chain
     def func():
         ...
    ```
### Runnable 로 정의 하는 함수 정의
- 이전 Chain의 출력을 입력 받는 파라미터를 한개 선언한다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
```python
def plus(num1, num2):
    ...

def wrapper_plus(nums:dict|list):
    return plus(nums['num1'], nums['num2'])
```
- Chain의 실행결과를 return 한다.

## Chain 간의 연결

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
    - https://python.langchain.com/docs/integrations/llms/llm_caching
```python
set_llm_cache(Cache객체)
```